<a href="https://colab.research.google.com/github/nickwan/nwds-stream-notebooks/blob/main/pybaseball_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install panel -q
!pip install pybaseball -q
!pip install adjustText -q

  Created wheel for pybaseball: filename=pybaseball-1.0.8-cp36-none-any.whl size=36008 sha256=66b5a910678feb443776d704222f61a8706982e36f124e320a16de11bcd2ec48
  Stored in directory: /root/.cache/pip/wheels/d5/7b/fb/b1f8a38eb135628a747627977fd18d13ce2c210cffa08ee5a5
Successfully built pybaseball


In [ ]:
import pybaseball as bb
import warnings
from bs4 import BeautifulSoup
import time
from joblib import dump, load
import random
import requests
import json
import datetime
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from adjustText import adjust_text
from tqdm.notebook import tqdm

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
def validate_datestring(date_text):
    try:
        datetime.datetime.strptime(date_text, '%Y-%m-%d')
    except ValueError:
        raise ValueError("Incorrect data format, should be YYYY-MM-DD")

def sanitize_input(start_dt, end_dt):
    # if no dates are supplied, assume they want yesterday's data
    # send a warning in case they wanted to specify
    if start_dt is None and end_dt is None:
        today = datetime.datetime.today()
        start_dt = (today - datetime.timedelta(1)).strftime("%Y-%m-%d")
        end_dt = today.strftime("%Y-%m-%d")
        print("Warning: no date range supplied. Returning yesterday's data. For a different date range, try batting_stats_range(start_dt, end_dt) or batting_stats(season).")

    #if only one date is supplied, assume they only want that day's stats
    #query in this case is from date 1 to date 1
    if start_dt is None:
        start_dt = end_dt
    if end_dt is None:
        end_dt = start_dt
    #if end date occurs before start date, swap them
    if end_dt < start_dt:
        temp = start_dt
        start_dt = end_dt
        end_dt = temp

    # now that both dates are not None, make sure they are valid date strings
    validate_datestring(start_dt)
    validate_datestring(end_dt)
    return start_dt, end_dt

def batting_stats_range(start_dt=None, end_dt=None):
    """
    Get all batting stats for a set time range. This can be the past week, the
    month of August, anything. Just supply the start and end date in YYYY-MM-DD
    format.
    """
    warnings.warn("\nPlease consider supporting FanGraphs\nhttps://plus.fangraphs.com/product/fangraphs-membership/", Warning)

    # make sure date inputs are valid
    start_dt, end_dt = sanitize_input(start_dt, end_dt)
    if datetime.datetime.strptime(start_dt, "%Y-%m-%d").year < 2008:
        raise ValueError("Year must be 2008 or later")
    if datetime.datetime.strptime(end_dt, "%Y-%m-%d").year < 2008:
        raise ValueError("Year must be 2008 or later")
    # retrieve html from baseball reference
    url = f"https://www.fangraphs.com/leaders.aspx?pos=all&stats=bat&lg=all&qual=1&type=c,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,34,35,36,37,38,39,40,60,41,201,205,200,52,51,50,61,62,63,64,65,66,67,68,69,70,71,53,111,54,56,203,199,204,55,57,58,59&season={start_dt[:4]}&month=1000&season1={end_dt[:4]}&ind=0&startdate={start_dt}&enddate={end_dt}&page=1_1500"
    s = requests.get(url).content
    soup = BeautifulSoup(s, "lxml")
    table = soup.find_all('table')[16]
    data = []
    headings = [th.get_text() for th in table.find_all("th")]
    data.append(headings)
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols])
    data = pd.DataFrame(data)
    data = data.rename(columns=data.iloc[0])
    table = data.reindex(data.index.drop(0))
    table = table.dropna(how='all')  # drop if all columns are NA
    # scraped data is initially in string format.
    # convert the necessary columns to numeric.
    for column in ['#', 'Age', 'G', 'AB', 'PA', 'H', '1B', '2B', '3B',
           'HR', 'R', 'RBI', 'BB', 'IBB', 'SO', 'HBP', 'SF', 'SH', 'GDP',
           'SB', 'CS', 'AVG', 'BB/K', 'OBP', 'SLG', 'OPS', 'ISO',
           'Spd', 'BABIP', 'UBR', 'wGDP', 'wSB', 'wRC', 'wRAA', 'wOBA',
           'wRC+', 'WPA', '-WPA', '+WPA', 'RE24', 'REW', 'pLI', 'phLI', 'PH',
           'WPA/LI', 'Clutch', 'Bat', 'BsR', 'Fld', 'Pos', 'Off', 'Def', 'Lg',
           'Rep', 'RAR', 'WAR']:
        #table[column] = table[column].astype('float')
        table[column] = pd.to_numeric(table[column])
        #table['column'] = table['column'].convert_objects(convert_numeric=True)
    table = table.reset_index(drop=True)
    return table

def pitching_stats_range(start_dt=None, end_dt=None):
    """
    Get all pitching stats for a set time range. This can be the past week, the
    month of August, anything. Just supply the start and end date in YYYY-MM-DD
    format.
    """
    warnings.warn("\nPlease consider supporting FanGraphs\nhttps://plus.fangraphs.com/product/fangraphs-membership/", Warning)
    # ensure valid date strings, perform necessary processing for query
    start_dt, end_dt = sanitize_input(start_dt, end_dt)
    if datetime.datetime.strptime(start_dt, "%Y-%m-%d").year < 2008:
        raise ValueError("Year must be 2008 or later")
    if datetime.datetime.strptime(end_dt, "%Y-%m-%d").year < 2008:
        raise ValueError("Year must be 2008 or later")
    # retrieve html from baseball reference
    # get most recent standings if date not specified
    if((start_dt is None) or (end_dt is None)):
        print('Error: a date range needs to be specified')
        return None
    url = f"https://www.fangraphs.com/leaders.aspx?pos=all&stats=pit&lg=all&qual=1&type=c,4,5,6,7,8,9,10,11,114,12,13,14,15,16,17,18,19,20,21,22,23,24,36,37,38,40,120,121,217,41,42,43,44,117,118,119,45,124,62,122,229,240,251,262,273,230,241,252,263,274,231,242,253,264,275,234,245,256,267,278,226,237,248,259,270,235,246,257,268,279,228,239,250,261,272,227,238,249,260,271,232,243,254,265,276,233,244,255,266,277,236,247,258,269,280,63,64,65,66,67,68,69,70,71,72,73,74,115,116,212,213,214,215,58,59,60,54,55,56,57,29,30,31,292,293,294,295,296,297,298,299&season={start_dt[:4]}&month=0&season1={end_dt[:4]}&ind=0&team=0&rost=0&age=0&filter=&players=0&startdate={start_dt}&enddate={end_dt}&page=1_1500"
    s = requests.get(url).content
    soup = BeautifulSoup(s, "lxml")

    table = soup.find_all('table')[16]
    data = []
    headings = [th.get_text() for th in table.find_all("th")]
    data.append(headings)
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols])
    data = pd.DataFrame(data)
    data = data.rename(columns=data.iloc[0])
    data = data.reindex(data.index.drop(0))
    table = data
    table = table.dropna(how='all')  # drop if all columns are NA
    # scraped data is initially in string format.
    # convert the necessary columns to numeric.
    for column in ['#', 'W', 'L', 'ERA', 'G', 'GS', 'CG', 'ShO', 'SV',
           'HLD', 'BS', 'IP', 'TBF', 'H', 'R', 'ER', 'HR', 'BB', 'IBB', 'HBP',
           'WP', 'BK', 'SO', 'K/9', 'BB/9', 'K/BB', 'HR/9', 'AVG', 'WHIP', 'BABIP',
           'ERA-', 'FIP-', 'xFIP-',
           'FIP', 'E-F', 'xFIP', 'SIERA', 'vFA (pi)', 'FA-X (pi)',
           'FA-Z (pi)', 'wFA (pi)', 'vFC (pi)', 'FC-X (pi)',
           'FC-Z (pi)', 'wFC (pi)', 'vFS (pi)', 'FS-X (pi)',
           'FS-Z (pi)', 'wFS (pi)', 'vSI (pi)', 'SI-X (pi)',
           'SI-Z (pi)', 'wSI (pi)', 'vCH (pi)', 'CH-X (pi)',
           'CH-Z (pi)', 'wCH (pi)', 'vSL (pi)', 'SL-X (pi)',
           'SL-Z (pi)', 'wSL (pi)', 'vCU (pi)', 'CU-X (pi)',
           'CU-Z (pi)', 'wCU (pi)', 'vCS (pi)', 'CS-X (pi)',
           'CS-Z (pi)', 'wCS (pi)', 'vKN (pi)', 'KN-X (pi)',
           'KN-Z (pi)', 'wKN (pi)', 'vSB (pi)', 'SB-X (pi)',
           'SB-Z (pi)', 'wSB (pi)', 'vXX (pi)', 'XX-X (pi)',
           'XX-Z (pi)', 'wXX (pi)', 'WPA', '-WPA', '+WPA', 'RE24', 'REW',
           'pLI', 'inLI', 'gmLI', 'exLI', 'Pulls', 'WPA/LI', 'Clutch', 'SD',
           'MD', 'RA9-WAR', 'BIP-Wins', 'LOB-Wins', 'FDP-Wins', 'RAR', 'WAR',
           'Starting', 'Start-IP', 'Relieving', 'Relief-IP',
           'Balls', 'Strikes', 'Pitches', 'Pace (pi)']:
        #table[column] = table[column].astype('float')
        table[column] = pd.to_numeric(table[column])
        #table['column'] = table['column'].convert_objects(convert_numeric=True)
    table = table.reset_index(drop=True)
    return table


In [ ]:
season = 2019
teams = bb.teams()
teams = teams.loc[teams['yearID']==season]

In [ ]:
schedule = pd.DataFrame()
for team in teams['teamIDBR'].unique():
  _schedule = bb.schedule_and_record(season=season,team=team)
  _schedule = _schedule.reset_index().rename(columns={'index':'game_num'})
  _schedule = _schedule.loc[_schedule['game_num']<=60]
  schedule = schedule.append(_schedule,ignore_index=True)
schedule['date_played'] = pd.to_datetime(schedule['Date'].str.split(', ',expand=True)[1].str.split('(',expand=True)[0].add(f' {season}'))
schedule['date_scheduled'] = pd.to_datetime(schedule['Orig. Scheduled'].str.split('(', expand=True)[0])
schedule['date_scheduled'] = schedule['date_scheduled'].fillna(schedule['date_played'])
schedule['GB'] = schedule['GB'].str.replace('Tied', '0').str.replace('up ', 'up').str.replace('up', '-').astype(float).mul(-1)
schedule = schedule.merge(schedule['W-L']
                          .str
                          .split('-',expand=True)
                          .rename(columns={0:'win_record',
                                           1:'loss_record'}),
                          left_index=True,
                          right_index=True)
int_cols = ['win_record','loss_record']
schedule.loc[:,int_cols] = schedule.loc[:,int_cols].astype(int)
schedule.shape

(1800, 24)

In [ ]:
max_date = schedule['date_played'].max().strftime("%Y-%m-%d")
min_date = schedule['date_played'].min().strftime("%Y-%m-%d")

In [ ]:
batters = batting_stats_range(min_date, max_date)
pitchers = pitching_stats_range(min_date, max_date)

print(f"batters: {batters.shape}\npitchers: {pitchers.shape}")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: Warning: 
Please consider supporting FanGraphs
https://plus.fangraphs.com/product/fangraphs-membership/
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:86: Warning: 
Please consider supporting FanGraphs
https://plus.fangraphs.com/product/fangraphs-membership/


batters: (730, 61)
pitchers: (818, 134)


In [ ]:
schedule.columns.values

array(['game_num', 'Date', 'Tm', 'Home_Away', 'Opp', 'W/L', 'R', 'RA',
       'Inn', 'W-L', 'Rank', 'GB', 'Win', 'Loss', 'Save', 'Time', 'D/N',
       'Attendance', 'Streak', 'Orig. Scheduled', 'date_played',
       'date_scheduled', 'win_record', 'loss_record'], dtype=object)

In [ ]:
schedule['game_num'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60])

In [ ]:
tex = bb.schedule_and_record(season=2019,team='TEX')
tex

,Date,Tm,Home_Away,Opp,W/L,R,RA,Inn,W-L,Rank,GB,Win,Loss,Save,Time,D/N,Attendance,Streak,Orig. Scheduled
1,"Thursday, Mar 28",TEX,Home,CHC,L,4.0,12.0,9.0,0-1,3.0,2.0,Lester,Minor,None,3:07,D,48538.0,-1,None
2,"Saturday, Mar 30",TEX,Home,CHC,W,8.0,6.0,9.0,1-1,2.0,1.5,Kelley,Edwards,Leclerc,3:47,N,46238.0,1,None
3,"Sunday, Mar 31",TEX,Home,CHC,W-wo,11.0,10.0,9.0,2-1,2.0,1.5,Leclerc,Strop,None,3:46,D,36812.0,2,None
4,"Monday, Apr 1",TEX,Home,HOU,L,1.0,2.0,9.0,2-2,3.0,2.5,Peacock,Sampson,Osuna,2:39,N,18056.0,-1,None
5,"Tuesday, Apr 2",TEX,Home,HOU,W,6.0,4.0,9.0,3-2,3.0,2.5,Kelley,Valdez,Leclerc,3:23,N,17907.0,1,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,"Wednesday, Sep 25",TEX,Home,BOS,L,3.0,10.0,9.0,75-83,3.0,29.0,Porcello,Allard,None,3:18,N,29290.0,-2,None
159,"Thursday, Sep 26",TEX,Home,BOS,W,7.0,5.0,9.0,76-83,3.0,28.0,Minor,Weber,Leclerc,3:02,D,24612.0,1,None
160,"Friday, Sep 27",TEX,Home,NYY,L,7.0,14.0,9.0,76-84,3.0,29.0,Tarpley,Palumbo,None,3:37,N,35168.0,-1,None
161,"Saturday, Sep 28",TEX,Home,NYY,W,9.0,4.0,9.0,77-84,3.0,29.0,Hernandez,Severino,None,3:17,N,42870.0,1,None


In [ ]:
last_game = schedule.loc[schedule['game_num']==60]
last_game = last_game.sort_values(['Rank','GB','win_record','loss_record'],ascending=[1,0,0,1])
last_game.head(10)

,game_num,Date,Tm,Home_Away,Opp,W/L,R,RA,Inn,W-L,Rank,GB,Win,Loss,Save,Time,D/N,Attendance,Streak,Orig. Scheduled,date_played,date_scheduled,win_record,loss_record
1019,60,"Wednesday, Jun 5",MIN,@,CLE,L,7.0,9.0,9.0,40-20,1.0,9.5,Wittgren,Parker,Hand,3:26,N,16962.0,-2,None,2019-06-05,2019-06-05,40,20
839,60,"Sunday, Jun 2",LAD,Home,PHI,W,8.0,0.0,9.0,41-19,1.0,9.0,Hill,Velasquez,None,2:58,D,49162.0,5,None,2019-06-02,2019-06-02,41,19
659,60,"Sunday, Jun 2",HOU,@,OAK,W,6.0,4.0,12.0,40-20,1.0,8.5,James,Trivino,None,4:24,D,23144.0,3,None,2019-06-02,2019-06-02,40,20
1079,60,"Wednesday, Jun 5",NYY,@,TOR,L,7.0,11.0,9.0,38-22,1.0,1.5,Gaviglio,Britton,None,3:22,N,16609.0,-3,None,2019-06-05,2019-06-05,38,22
959,60,"Sunday, Jun 2",MIL,@,PIT,W,4.0,2.0,9.0,34-26,1.0,1.5,Davies,Lyles,Burnes,2:41,D,19442.0,2,None,2019-06-02,2019-06-02,34,26
359,60,"Wednesday, Jun 5",CHC,Home,COL,W,9.0,8.0,9.0,34-26,1.0,1.0,Kintzler,Marquez,Cishek,3:22,N,35395.0,3,None,2019-06-05,2019-06-05,34,26
1259,60,"Monday, Jun 3",PHI,@,SDP,L,2.0,8.0,9.0,33-27,1.0,0.5,Lauer,Nola,None,2:54,N,21654.0,-5,None,2019-06-03,2019-06-03,33,27
119,60,"Tuesday, Jun 4",ATL,@,PIT,W,12.0,5.0,9.0,33-27,2.0,-0.5,Winkler,Crick,None,3:46,N,13963.0,3,None,2019-06-04,2019-06-04,33,27
1619,60,"Thursday, Jun 6",TBR,@,DET,W,6.0,1.0,9.0,37-23,2.0,-1.5,Beeks,Norris,None,3:08,D,21442.0,2,None,2019-06-06,2019-06-06,37,23
1679,60,"Thursday, Jun 6",TEX,Home,BAL,W,4.0,3.0,9.0,32-28,2.0,-9.0,Jurado,Hess,Kelley,2:29,N,20462.0,2,None,2019-06-06,2019-06-06,32,28


In [ ]:
batters.sort_values('HR',ascending=False).head(10)

,#,Name,Team,Age,G,AB,PA,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG,BB%,K%,BB/K,OBP,SLG,OPS,ISO,Spd,BABIP,UBR,wGDP,wSB,wRC,wRAA,wOBA,wRC+,WPA,-WPA,+WPA,RE24,REW,pLI,phLI,PH,WPA/LI,Clutch,Bat,BsR,Fld,Pos,Off,Def,Lg,Rep,RAR,WAR,Dol
20,21,Christian Yelich,Brewers,27,56,201,244,66,33,8,2,23,47,51,37,12,40,4,2,0,5,13,1,0.328,15.2%,16.4%,0.93,0.439,0.731,1.170,0.403,6.8,0.307,1.3,0.2,2.0,60,29.1,0.458,185,2.74,-3.33,6.07,28.98,2.87,0.91,2.66,2,3.03,-0.02,27.3,3.6,0.8,-2.5,30.9,-1.6,0.2,7.7,37.1,3.6,$28.9
62,63,Peter Alonso,Mets,24,61,223,253,59,25,12,2,20,36,45,20,0,68,8,2,0,5,0,0,0.265,7.9%,26.9%,0.29,0.344,0.605,0.949,0.341,3.0,0.285,-0.8,0.0,-0.1,46,14.5,0.386,144,1.30,-4.40,5.71,14.98,1.61,1.00,0.82,5,1.41,-0.10,14.8,-0.9,0.6,-4.2,14.0,-3.6,0.2,8.0,18.6,1.8,$14.4
17,18,Cody Bellinger,Dodgers,23,60,219,259,81,46,13,2,20,51,54,36,7,36,1,3,0,2,7,5,0.370,13.9%,13.9%,1.00,0.456,0.721,1.177,0.352,5.0,0.367,0.0,0.4,-0.9,66,33.6,0.470,198,3.31,-3.52,6.83,36.03,3.63,0.92,0.47,1,3.88,-0.28,33.4,-0.6,3.8,-2.5,32.9,1.3,0.2,8.1,42.6,4.1,$33.1
57,58,Gary Sanchez,Yankees,26,43,164,182,43,20,3,1,19,27,37,15,1,50,3,0,0,1,0,0,0.262,8.2%,27.5%,0.30,0.335,0.640,0.975,0.378,2.3,0.253,-0.8,0.6,-0.1,35,11.6,0.394,149,0.23,-3.56,3.79,7.06,0.71,1.02,2.88,2,1.40,-1.17,11.3,-0.3,-2.0,1.9,11.0,0.0,0.7,5.7,17.4,1.7,$13.5
146,147,Franmil Reyes,Padres,23,60,196,215,48,22,7,0,19,29,32,15,0,58,0,4,0,4,0,0,0.245,7.0%,27.0%,0.26,0.293,0.571,0.864,0.327,0.9,0.236,-0.2,-0.9,-0.1,32,5.3,0.348,117,0.34,-3.95,4.29,-1.43,-0.13,0.95,1.69,8,0.99,-0.63,4.7,-1.1,-2.3,-2.1,3.6,-4.4,0.2,6.8,6.2,0.6,$4.8
48,49,Joc Pederson,Dodgers,27,56,172,201,45,20,4,3,18,41,33,22,0,40,6,1,0,2,0,0,0.262,10.9%,19.9%,0.55,0.363,0.634,0.997,0.372,5.0,0.235,0.0,0.2,-0.1,40,14.7,0.404,155,1.31,-3.15,4.46,14.18,1.55,0.88,0.76,14,1.53,-0.03,14.6,0.1,0.7,-2.2,14.7,-1.5,0.2,6.3,19.7,1.9,$15.3
94,95,Mike Moustakas,Brewers,30,56,221,241,59,27,14,0,18,40,39,17,0,48,3,0,0,7,3,0,0.267,7.1%,19.9%,0.35,0.328,0.575,0.902,0.308,4.3,0.265,-1.1,0.1,0.5,41,10.6,0.371,128,0.66,-4.17,4.83,13.11,1.36,0.93,0.03,2,1.86,-1.15,8.8,-0.5,0.1,0.8,8.3,0.8,0.2,7.6,16.9,1.6,$13.1
93,94,Hunter Renfroe,Padres,27,57,176,191,44,16,9,1,18,26,35,14,0,52,0,1,0,3,3,0,0.250,7.3%,27.2%,0.27,0.304,0.619,0.923,0.369,4.8,0.243,-0.6,0.2,0.5,33,8.5,0.372,132,0.73,-3.78,4.50,8.33,0.86,1.04,2.04,9,0.67,0.02,8.1,0.1,3.9,-1.9,8.1,2.0,0.2,6.0,16.3,1.6,$12.7
141,142,Eddie Rosario,Twins,27,59,235,250,64,37,8,1,18,42,50,13,0,37,0,2,0,7,2,1,0.272,5.2%,14.8%,0.35,0.308,0.545,0.853,0.272,4.3,0.253,-0.2,0.3,-0.1,38,6.4,0.349,117,0.85,-3.82,4.68,6.80,0.69,0.89,NaN,0,0.39,0.57,5.6,0.0,-2.4,-2.6,5.5,-5.0,1.0,7.9,9.4,0.9,$7.3
23,24,Josh Bell,Pirates,26,60,234,261,79,34,25,2,18,45,56,24,4,54,1,2,0,5,0,1,0.338,9.2%,20.7%,0.44,0.398,0.692,1.091,0.355,3.3,0.372,-1.9,0.0,-0.6,59,26.6,0.438,174,2.28,-3.97,6.25,23.54,2.35,0.95,NaN,0,2.34,0.05,25.6,-2.4,-3.2,-4.6,23.2,-7.8,0.2,8.2,23.9,2.3,$18.6


In [ ]:
batters.sort_values('WAR',ascending=False).head(10)

,#,Name,Team,Age,G,AB,PA,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG,BB%,K%,BB/K,OBP,SLG,OPS,ISO,Spd,BABIP,UBR,wGDP,wSB,wRC,wRAA,wOBA,wRC+,WPA,-WPA,+WPA,RE24,REW,pLI,phLI,PH,WPA/LI,Clutch,Bat,BsR,Fld,Pos,Off,Def,Lg,Rep,RAR,WAR,Dol
17,18,Cody Bellinger,Dodgers,23,60,219,259,81,46,13,2,20,51,54,36,7,36,1,3,0,2,7,5,0.370,13.9%,13.9%,1.00,0.456,0.721,1.177,0.352,5.0,0.367,0.0,0.4,-0.9,66,33.6,0.470,198,3.31,-3.52,6.83,36.03,3.63,0.92,0.47,1,3.88,-0.28,33.4,-0.6,3.8,-2.5,32.9,1.3,0.2,8.1,42.6,4.1,$33.1
34,35,Mike Trout,Angels,27,59,198,264,58,28,14,0,16,46,41,57,10,43,7,2,0,4,7,1,0.293,21.6%,16.3%,1.33,0.462,0.606,1.068,0.313,4.1,0.298,1.6,1.1,0.8,59,25.7,0.433,177,2.60,-3.51,6.10,32.81,3.22,0.95,NaN,0,3.07,-0.35,26.0,3.5,-0.4,0.2,29.5,-0.2,1.0,8.3,38.7,3.8,$30.1
20,21,Christian Yelich,Brewers,27,56,201,244,66,33,8,2,23,47,51,37,12,40,4,2,0,5,13,1,0.328,15.2%,16.4%,0.93,0.439,0.731,1.170,0.403,6.8,0.307,1.3,0.2,2.0,60,29.1,0.458,185,2.74,-3.33,6.07,28.98,2.87,0.91,2.66,2,3.03,-0.02,27.3,3.6,0.8,-2.5,30.9,-1.6,0.2,7.7,37.1,3.6,$28.9
35,36,Joey Gallo,Rangers,25,50,170,214,47,18,11,1,17,41,41,42,4,76,1,1,0,0,3,0,0.276,19.6%,35.5%,0.55,0.421,0.653,1.074,0.376,5.5,0.385,1.2,0.8,0.5,47,20.5,0.431,164,0.79,-3.54,4.33,13.33,1.08,1.02,NaN,0,0.90,-0.12,17.6,2.5,3.7,-1.1,20.1,2.7,0.8,6.7,30.4,2.9,$23.6
41,42,George Springer,Astros,29,48,185,216,57,30,9,1,17,41,43,25,0,45,2,4,0,3,4,1,0.308,11.6%,20.8%,0.56,0.389,0.643,1.032,0.335,5.4,0.315,0.8,-0.4,0.2,45,18.3,0.418,168,1.18,-3.43,4.61,14.90,1.41,0.94,2.17,1,1.56,-0.32,18.7,0.6,3.4,-0.9,19.3,2.5,0.8,6.8,29.5,2.9,$22.9
52,53,Alex Bregman,Astros,25,62,224,275,63,35,10,0,18,43,44,43,0,36,3,5,0,4,3,1,0.281,15.6%,13.1%,1.19,0.396,0.567,0.963,0.286,3.0,0.257,-1.4,0.5,0.0,53,18.5,0.398,154,1.61,-3.64,5.25,15.62,1.56,0.90,NaN,0,1.83,-0.04,19.1,-0.9,0.6,1.3,18.2,1.9,1.1,8.6,29.8,2.9,$23.2
98,99,Matt Chapman,Athletics,26,62,239,272,65,34,14,1,16,41,36,28,0,47,3,2,0,3,0,1,0.272,10.3%,17.3%,0.60,0.353,0.540,0.893,0.268,2.6,0.275,0.3,0.3,-0.6,46,11.8,0.370,136,1.11,-4.40,5.52,11.21,1.10,0.96,NaN,0,1.37,-0.21,12.4,0.0,6.0,0.9,12.4,6.9,1.1,8.6,29.0,2.8,$22.5
45,46,Nolan Arenado,Rockies,28,60,243,269,82,48,16,2,16,48,54,21,5,28,1,4,0,3,1,1,0.337,7.8%,10.4%,0.75,0.387,0.617,1.004,0.280,3.8,0.325,1.3,-0.1,-0.4,54,19.9,0.406,137,1.88,-4.32,6.20,24.62,2.16,1.03,NaN,0,1.45,0.38,13.2,0.9,4.2,0.9,14.1,5.1,0.3,8.5,27.9,2.7,$21.7
24,25,Anthony Rendon,Nationals,29,48,173,208,56,24,19,2,11,44,40,29,4,33,5,1,0,5,1,0,0.324,13.9%,15.9%,0.88,0.433,0.647,1.080,0.324,5.4,0.346,0.3,-0.2,0.1,47,21.1,0.437,170,1.37,-3.46,4.83,22.61,2.24,1.01,NaN,0,1.50,-0.14,19.3,0.1,0.6,0.7,19.4,1.3,0.2,6.5,27.5,2.7,$21.4
80,81,Javier Baez,Cubs,26,59,241,258,74,40,18,1,15,41,40,16,2,79,0,1,0,6,2,3,0.307,6.2%,30.6%,0.20,0.349,0.577,0.926,0.270,3.7,0.399,1.6,-0.5,-1.0,46,13.1,0.379,134,0.46,-4.90,5.37,6.39,0.59,1.02,5.55,1,1.16,-0.71,11.7,0.0,4.7,2.5,11.7,7.2,0.2,8.1,27.3,2.7,$21.2


In [ ]:
pitchers.sort_values('SO',ascending=False).head(10)

,#,Name,Team,W,L,ERA,G,GS,CG,ShO,SV,HLD,BS,IP,TBF,H,R,ER,HR,BB,IBB,HBP,WP,BK,SO,K/9,BB/9,K/BB,HR/9,K%,BB%,K-BB%,AVG,WHIP,BABIP,LOB%,ERA-,FIP-,xFIP-,FIP,...,vXX (pi),XX-X (pi),XX-Z (pi),wXX (pi),WPA,-WPA,+WPA,RE24,REW,pLI,inLI,gmLI,exLI,Pulls,WPA/LI,Clutch,SD,MD,RA9-WAR,BIP-Wins,LOB-Wins,FDP-Wins,RAR,WAR,Dollars,Starting,Start-IP,Relieving,Relief-IP,Balls,Strikes,Pitches,O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi)
93,94,Gerrit Cole,Astros,20,5,2.50,33,33,0,0,0,0,0,212.1,817,142,66,59,29,48,0,3,4,3,326,13.82,2.03,6.79,1.23,39.9%,5.9%,34.0%,0.185,0.89,0.275,83.3%,56,59,55,2.64,...,NaN,NaN,NaN,NaN,4.31,-10.41,14.72,49.91,4.94,0.90,0.89,0.88,0.88,33,5.13,-0.34,0,0,7.8,0.9,-0.6,0.3,69.7,7.4,$59.3,69.7,212.1,NaN,NaN,1108,2254,3362,34.0%,64.6%,49.8%,48.1%,75.1%,66.2%,51.6%,22.9
95,96,Justin Verlander,Astros,21,6,2.58,34,34,2,1,0,0,0,223.0,847,137,66,64,36,42,0,6,4,0,300,12.11,1.70,7.14,1.45,35.4%,5.0%,30.5%,0.171,0.80,0.218,88.4%,58,73,70,3.27,...,NaN,NaN,NaN,NaN,5.19,-10.51,15.70,53.75,5.26,0.90,0.97,0.87,0.91,32,4.82,0.95,0,0,8.8,3.5,-1.1,2.4,61.4,6.4,$50.9,61.4,223.0,NaN,NaN,1087,2361,3448,37.1%,66.0%,51.9%,52.8%,77.4%,68.8%,51.0%,25.9
159,160,Shane Bieber,Indians,15,8,3.28,34,33,3,2,0,0,0,214.1,859,186,86,78,31,40,1,6,6,1,259,10.88,1.68,6.48,1.30,30.2%,4.7%,25.5%,0.229,1.05,0.296,77.4%,68,73,71,3.32,...,NaN,NaN,NaN,NaN,3.82,-13.07,16.89,40.87,3.77,0.96,0.97,0.85,1.25,30,3.29,0.68,0,0,6.0,0.1,0.3,0.4,54.8,5.6,$44.9,54.5,212.1,0.3,2.0,1124,2208,3332,35.2%,62.5%,47.7%,48.0%,85.4%,70.5%,45.9%,22.7
84,85,Jacob deGrom,Mets,11,8,2.43,32,32,0,0,0,0,0,204.0,804,154,59,55,19,44,1,7,2,0,255,11.25,1.94,5.80,0.84,31.7%,5.5%,26.2%,0.205,0.97,0.282,81.8%,59,61,70,2.67,...,NaN,NaN,NaN,NaN,4.21,-11.29,15.49,40.89,4.18,1.01,0.94,0.88,1.06,32,4.76,-0.60,0,0,7.7,0.6,0.1,0.7,64.9,7.0,$56.0,64.9,204.0,NaN,NaN,1075,2222,3297,36.5%,69.6%,52.9%,53.6%,79.1%,70.2%,49.5%,24.2
362,363,Trevor Bauer,- - -,11,13,4.48,34,34,1,1,0,0,0,213.0,911,184,118,106,34,82,0,19,10,0,253,10.69,3.46,3.09,1.44,27.8%,9.0%,18.8%,0.227,1.25,0.287,70.4%,96,95,96,4.34,...,79.8,1.7,27.9,0.0,0.38,-16.16,16.54,0.48,0.01,0.98,0.92,0.87,1.12,33,0.15,0.24,0,0,2.8,0.5,-1.0,-0.5,33.5,3.3,$26.2,33.5,212.3,NaN,NaN,1369,2318,3687,31.2%,61.5%,45.1%,53.6%,84.8%,73.1%,45.9%,24.1
163,164,Stephen Strasburg,Nationals,18,6,3.32,33,33,0,0,0,0,0,209.0,841,161,79,77,24,56,4,10,8,0,251,10.81,2.41,4.48,1.03,29.9%,6.7%,23.2%,0.208,1.04,0.274,76.5%,74,72,71,3.25,...,NaN,NaN,NaN,NaN,3.28,-12.34,15.62,30.79,3.10,0.98,0.95,0.88,1.07,33,3.25,0.10,0,0,6.4,0.9,-0.2,0.7,54.3,5.7,$45.4,54.3,209.0,NaN,NaN,1173,2211,3384,37.5%,59.2%,47.2%,55.8%,84.0%,71.6%,44.7%,26.3
213,214,Lance Lynn,Rangers,16,11,3.67,33,33,0,0,0,0,0,208.1,875,195,89,85,21,59,0,8,18,0,246,10.63,2.55,4.17,0.91,28.1%,6.7%,21.4%,0.241,1.22,0.322,74.4%,73,66,84,3.13,...,NaN,NaN,NaN,NaN,2.64,-12.80,15.44,35.40,3.08,0.92,0.88,0.87,0.80,33,1.77,1.09,0,0,6.1,-0.9,0.3,-0.6,64.2,6.8,$54.0,64.2,208.1,NaN,NaN,1275,2278,3553,28.3%,67.4%,48.6%,60.7%,80.0%,74.6%,51.8%,25.1
123,124,Max Scherzer,Nationals,11,7,2.92,27,27,0,0,0,0,0,172.1,693,144,59,56,18,33,2,7,0,0,243,12.69,1.72,7.36,0.94,35.1%,4.8%,30.3%,0.221,1.03,0.321,78.7%,65,54,65,2.45,...,NaN,NaN,NaN,NaN,3.47,-9.65,13.11,34.26,3.34,0.95,0.93,0.88,1.11,27,2.86,0.80,0,0,6.0,-0.9,0.4,-0.6,59.6,6.5,$52.2,59.6,172.1,NaN,NaN,858,1912,2770,36.0%,67.5%,52.2%,49.5%,77.9%,68.4%,51.6%,25.0
141,142,Charlie Morton,Rays,16,6,3.05,33,33,0,0,0,0,0,194.2,790,154,71,66,15,57,0,12,5,1,240,11.10,2.64,4.21,0.69,30.4%,7.2%,23.2%,0.214,1.08,0.298,75.3%,69,64,72,2.81,...,NaN,NaN,NaN,NaN,3.40,-12.23,15.63,33.79,3.31,1.04,0.97,0.87,1.16,33,3.67,-0.41,0,0,5.8,0.1,-0.4,-0.3,58.3,6.1,$48.5,58.3,194.2,NaN,NaN,1065,2074,3139,32.2%,61.4%,47.3%,52.3%,82.5%,72.6%,51.7%,22.9
156,157,Patrick Corbin,Nationals,14,7,3.25,33,33,1,1,0,0,0,202.0,835,169,81,73,24,70,2,3,4,0,238,10.60,3.12,3.40,1.07,28.5%,8.4%,20.1%,0.222,1.18,0.290,77.3%,72,77,81,3.49,...,NaN,NaN,NaN,NaN,2.60,-12.52,15.1

In [ ]:
pitchers.sort_values('WAR',ascending=False).head(10)

,#,Name,Team,W,L,ERA,G,GS,CG,ShO,SV,HLD,BS,IP,TBF,H,R,ER,HR,BB,IBB,HBP,WP,BK,SO,K/9,BB/9,K/BB,HR/9,K%,BB%,K-BB%,AVG,WHIP,BABIP,LOB%,ERA-,FIP-,xFIP-,FIP,...,vXX (pi),XX-X (pi),XX-Z (pi),wXX (pi),WPA,-WPA,+WPA,RE24,REW,pLI,inLI,gmLI,exLI,Pulls,WPA/LI,Clutch,SD,MD,RA9-WAR,BIP-Wins,LOB-Wins,FDP-Wins,RAR,WAR,Dollars,Starting,Start-IP,Relieving,Relief-IP,Balls,Strikes,Pitches,O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi)
93,94,Gerrit Cole,Astros,20,5,2.50,33,33,0,0,0,0,0,212.1,817,142,66,59,29,48,0,3,4,3,326,13.82,2.03,6.79,1.23,39.9%,5.9%,34.0%,0.185,0.89,0.275,83.3%,56,59,55,2.64,...,NaN,NaN,NaN,NaN,4.31,-10.41,14.72,49.91,4.94,0.90,0.89,0.88,0.88,33,5.13,-0.34,0,0,7.8,0.9,-0.6,0.3,69.7,7.4,$59.3,69.7,212.1,NaN,NaN,1108,2254,3362,34.0%,64.6%,49.8%,48.1%,75.1%,66.2%,51.6%,22.9
84,85,Jacob deGrom,Mets,11,8,2.43,32,32,0,0,0,0,0,204.0,804,154,59,55,19,44,1,7,2,0,255,11.25,1.94,5.80,0.84,31.7%,5.5%,26.2%,0.205,0.97,0.282,81.8%,59,61,70,2.67,...,NaN,NaN,NaN,NaN,4.21,-11.29,15.49,40.89,4.18,1.01,0.94,0.88,1.06,32,4.76,-0.60,0,0,7.7,0.6,0.1,0.7,64.9,7.0,$56.0,64.9,204.0,NaN,NaN,1075,2222,3297,36.5%,69.6%,52.9%,53.6%,79.1%,70.2%,49.5%,24.2
213,214,Lance Lynn,Rangers,16,11,3.67,33,33,0,0,0,0,0,208.1,875,195,89,85,21,59,0,8,18,0,246,10.63,2.55,4.17,0.91,28.1%,6.7%,21.4%,0.241,1.22,0.322,74.4%,73,66,84,3.13,...,NaN,NaN,NaN,NaN,2.64,-12.80,15.44,35.40,3.08,0.92,0.88,0.87,0.80,33,1.77,1.09,0,0,6.1,-0.9,0.3,-0.6,64.2,6.8,$54.0,64.2,208.1,NaN,NaN,1275,2278,3553,28.3%,67.4%,48.6%,60.7%,80.0%,74.6%,51.8%,25.1
123,124,Max Scherzer,Nationals,11,7,2.92,27,27,0,0,0,0,0,172.1,693,144,59,56,18,33,2,7,0,0,243,12.69,1.72,7.36,0.94,35.1%,4.8%,30.3%,0.221,1.03,0.321,78.7%,65,54,65,2.45,...,NaN,NaN,NaN,NaN,3.47,-9.65,13.11,34.26,3.34,0.95,0.93,0.88,1.11,27,2.86,0.80,0,0,6.0,-0.9,0.4,-0.6,59.6,6.5,$52.2,59.6,172.1,NaN,NaN,858,1912,2770,36.0%,67.5%,52.2%,49.5%,77.9%,68.4%,51.6%,25.0
95,96,Justin Verlander,Astros,21,6,2.58,34,34,2,1,0,0,0,223.0,847,137,66,64,36,42,0,6,4,0,300,12.11,1.70,7.14,1.45,35.4%,5.0%,30.5%,0.171,0.80,0.218,88.4%,58,73,70,3.27,...,NaN,NaN,NaN,NaN,5.19,-10.51,15.70,53.75,5.26,0.90,0.97,0.87,0.91,32,4.82,0.95,0,0,8.8,3.5,-1.1,2.4,61.4,6.4,$50.9,61.4,223.0,NaN,NaN,1087,2361,3448,37.1%,66.0%,51.9%,52.8%,77.4%,68.8%,51.0%,25.9
141,142,Charlie Morton,Rays,16,6,3.05,33,33,0,0,0,0,0,194.2,790,154,71,66,15,57,0,12,5,1,240,11.10,2.64,4.21,0.69,30.4%,7.2%,23.2%,0.214,1.08,0.298,75.3%,69,64,72,2.81,...,NaN,NaN,NaN,NaN,3.40,-12.23,15.63,33.79,3.31,1.04,0.97,0.87,1.16,33,3.67,-0.41,0,0,5.8,0.1,-0.4,-0.3,58.3,6.1,$48.5,58.3,194.2,NaN,NaN,1065,2074,3139,32.2%,61.4%,47.3%,52.3%,82.5%,72.6%,51.7%,22.9
163,164,Stephen Strasburg,Nationals,18,6,3.32,33,33,0,0,0,0,0,209.0,841,161,79,77,24,56,4,10,8,0,251,10.81,2.41,4.48,1.03,29.9%,6.7%,23.2%,0.208,1.04,0.274,76.5%,74,72,71,3.25,...,NaN,NaN,NaN,NaN,3.28,-12.34,15.62,30.79,3.10,0.98,0.95,0.88,1.07,33,3.25,0.10,0,0,6.4,0.9,-0.2,0.7,54.3,5.7,$45.4,54.3,209.0,NaN,NaN,1173,2211,3384,37.5%,59.2%,47.2%,55.8%,84.0%,71.6%,44.7%,26.3
159,160,Shane Bieber,Indians,15,8,3.28,34,33,3,2,0,0,0,214.1,859,186,86,78,31,40,1,6,6,1,259,10.88,1.68,6.48,1.30,30.2%,4.7%,25.5%,0.229,1.05,0.296,77.4%,68,73,71,3.32,...,NaN,NaN,NaN,NaN,3.82,-13.07,16.89,40.87,3.77,0.96,0.97,0.85,1.25,30,3.29,0.68,0,0,6.0,0.1,0.3,0.4,54.8,5.6,$44.9,54.5,212.1,0.3,2.0,1124,2208,3332,35.2%,62.5%,47.7%,48.0%,85.4%,70.5%,45.9%,22.7
126,127,Zack Greinke,- - -,18,5,2.93,33,33,0,0,0,0,0,208.2,810,175,73,68,21,30,2,4,2,1,187,8.07,1.29,6.23,0.91,23.1%,3.7%,19.4%,0.226,0.98,0.271,75.7%,66,73,84,3.22,...,77.0,-7.3,-2.3,0.0,3.36,-12.00,15.36,37.51,3.61,0.95,0.93,0.88,1.08,33,3.91,-0.38,0,0,6.7,1.2,0.0,1.3,52.3,5.4,$43.3,52.3,208.2,NaN,NaN,1081,2032,3113,34.6%,60.3%,46.5%,64.7%,85.7%,77.2%,46.1%,25.5
178,179,Lucas Giolito,White Sox,14,9,3.41,29,29,3,2,0,0,0,176.2,705,131,69,67,24,57,1,4,6,0,228,11.62,2.90,4.00,1.22,32.3%,8.1%,24.3%,0.203,1.06,0.273,77.7%,76,74,80,3.43,...,NaN,NaN,NaN,NaN,2.74,-11.15,13.89,25.7

In [ ]:
perc_cols = ['BB%', 'K%']
for col in perc_cols:
  batters.loc[:,col] = batters.loc[:,col].str.rstrip('%').astype(float).div(100)

In [ ]:
app_cols = ['Name','Team','PA','AVG','K%','BB%','OBP','SLG','OPS','wOBA',
            'wRC+','WPA/LI','RAR','WAR']
app_data = batters.loc[:,app_cols]

In [ ]:
app_data

,Name,Team,PA,AVG,K%,BB%,OBP,SLG,OPS,wOBA,wRC+,WPA/LI,RAR,WAR
248,Manny Machado,Padres,261,0.246,0.215,0.107,0.333,0.412,0.746,0.318,97,0.39,8.5,0.8
681,Randy Rosario,Cubs,1,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-100,0.00,-0.1,0.0
226,Tim Beckham,Mariners,200,0.241,0.310,0.060,0.290,0.487,0.777,0.322,104,-0.15,3.1,0.3
556,Julio Teheran,Braves,24,0.130,0.458,0.000,0.130,0.174,0.304,0.129,-31,-0.39,-0.4,0.0
714,Mitch Keller,Pirates,1,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-100,-0.03,-0.1,0.0
